In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

/home/eclipse/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
PATH = '../datafiles/'

In [4]:
!ls {PATH}

'adult (copy).csv'   gender_submission.csv   student-mat.csv
 adult.data	     housing.csv	     test.csv
 bluebook	     melb_data.csv	     train.csv
 citibike.csv	     oecd.csv		     WEO_Data.csv
 fifa.csv	     ram_price.csv	     winequality-red.csv


In [5]:
train_df_raw = pd.read_csv(f'{PATH}train.csv', sep=',', low_memory=False)
test_df_raw = pd.read_csv(f'{PATH}test.csv', sep=',')

In [6]:
def shape(dataframe):
    return print('Dataset shape :\n{}'.format(dataframe.shape))

def display_transpose(dataframe):
    pd.option_context('display.max_rows' ,1000, 'display.max_columns', 1000)
    return display(dataframe.transpose())

In [7]:
train_df_raw[:20]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [8]:
#transposed view of the dataset
display_transpose(train_df_raw[:20])
shape(train_df_raw)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
PassengerId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Survived,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,1,0,1,0,1
Pclass,3,1,3,1,3,3,1,3,3,2,3,1,3,3,3,2,3,2,3,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry","Moran, Mr. James","McCarthy, Mr. Timothy J","Palsson, Master. Gosta Leonard","Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)","Nasser, Mrs. Nicholas (Adele Achem)","Sandstrom, Miss. Marguerite Rut","Bonnell, Miss. Elizabeth","Saundercock, Mr. William Henry","Andersson, Mr. Anders Johan","Vestrom, Miss. Hulda Amanda Adolfina","Hewlett, Mrs. (Mary D Kingcome)","Rice, Master. Eugene","Williams, Mr. Charles Eugene","Vander Planke, Mrs. Julius (Emelia Maria Vande...","Masselmani, Mrs. Fatima"
Sex,male,female,female,female,male,male,male,male,female,female,female,female,male,male,female,female,male,male,female,female
Age,22,38,26,35,35,NaN,54,2,27,14,4,58,20,39,14,55,2,NaN,31,NaN
SibSp,1,1,0,1,0,0,0,3,0,1,1,0,0,1,0,0,4,0,1,0
Parch,0,0,0,0,0,0,0,1,2,0,1,0,0,5,0,0,1,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450,330877,17463,349909,347742,237736,PP 9549,113783,A/5. 2151,347082,350406,248706,382652,244373,345763,2649
Fare,7.25,71.2833,7.925,53.1,8.05,8.4583,51.8625,21.075,11.1333,30.0708,16.7,26.55,8.05,31.275,7.8542,16,29.125,13,18,7.225


Dataset shape :
(891, 12)


In [11]:
#changing categorical varibles into numerics
train_cats(train_df_raw)

In [19]:
display_transpose(train_df_raw.isnull().sum().sort_index()/len(train_df_raw))

Age            0.198653
Cabin          0.771044
Embarked       0.002245
Fare           0.000000
Name           0.000000
Parch          0.000000
PassengerId    0.000000
Pclass         0.000000
Sex            0.000000
SibSp          0.000000
Survived       0.000000
Ticket         0.000000
dtype: float64

In [49]:
train_df, train_y, nas = proc_df(train_df_raw, 'Survived')

In [50]:
#trying a model without splitting 
titanic = RandomForestClassifier(n_jobs=-1).fit(train_df, train_y)
print('Training score : \n{}'.format(titanic.score(train_df, train_y)))

Training score : 
1.0


Splitting into training and validation sets

In [54]:
train_raw = train_df.copy()
raw_y = train_y.copy()
X_train, X_valid = (train_df[:600], train_df[600:])
y_train, y_valid = (train_y[:600], train_y[600:])

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((600, 12), (291, 12), (600,), (291,))

In [56]:
titanic = RandomForestClassifier(n_jobs=-1).fit(X_train, y_train)
print('Training score : \n{}'.format(titanic.score(X_train, y_train)))
print('Training score : \n{}'.format(titanic.score(X_valid, y_valid)))

Training score : 
1.0
Training score : 
0.8625429553264605
